<a href="https://colab.research.google.com/github/heincalitz/2025DE245Prac2/blob/main/2025DE245Prac2_ExtractLoad_Currency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [2]:
# Install required packages
!pip install requests

In [3]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [19]:
target_currency = input("Enter the target currency: ")

Enter the target currency: eur


In [20]:
# Read data from CoinGecko API
url = f"https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
response = requests.get(url)
data = response.json()

data
usd_amount = data["bitcoin"]["usd"]
usd_amount

114036

Obtain usd vs target currency exchange rate

In [21]:
url = f"https://api.coingecko.com/api/v3/simple/price?ids=usd&vs_currencies={target_currency}"
response2 = requests.get(url)
data2 = response2.json()
exchange_rate = data2["usd"][target_currency]
exchange_rate

0.857815

We will extract the Bitcoin price in USD and convert it to ZAR using an estimated exchange rate.
This is an example of turning raw data into useful **information**.

In [22]:
# Function to convert USD to Whatever curreny
def convert_usd_to_cur(usd_amount, rate):
    return round(usd_amount * rate, 2)

In [23]:
# Extract USD price and apply transformation
usd_price = data["bitcoin"]["usd"]
cur_price = convert_usd_to_cur(usd_price,exchange_rate)
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in USD: {usd_price}")
print(f"Estimated price in {target_currency}: {cur_price}")


Bitcoin price in USD: 114036
Estimated price in eur: 97821.79


In [24]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table with the new, required schema
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    timestamp TEXT PRIMARY KEY,
    bitcoin_price_usd REAL,
    target_currency TEXT,
    bitcoin_price_target_currency REAL
)
''')

# Insert the new row with values for all four columns
cursor.execute(
    "INSERT INTO prices VALUES (?, ?, ?, ?)",
    (timestamp, usd_amount, target_currency, cur_price)
)

conn.commit()
conn.close()


In [25]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()


('2025-08-19T15:48:20.817537', 114036.0, 'eur', 97821.79)
